# SQL query from table names - Continued

In [ ]:
!pip install python-dotenv


In [ ]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [ ]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [ ]:
context = [ {'role':'system', 'content':"""
You are a SQL assistant designed to generate **correct and efficient SQL queries** for a given database structure.
Below is the schema with table definitions and some example data.

### Database Schema:
```sql
CREATE TABLE products (
    product_id INTEGER PRIMARY KEY,
    product_name VARCHAR(100),
    category VARCHAR(50),
    price DECIMAL(10,2),
    stock_quantity INTEGER
);

CREATE TABLE customers (
    customer_id INTEGER PRIMARY KEY,
    customer_name VARCHAR(50),
    email VARCHAR(100),
    city VARCHAR(50),
    country VARCHAR(50)
);

CREATE TABLE orders (
    order_id INTEGER PRIMARY KEY,
    customer_id INTEGER,
    order_date DATE,
    total_price DECIMAL(10,2),
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
);

INSERT INTO products (product_id, product_name, category, price, stock_quantity) VALUES
(1, 'Laptop', 'Electronics', 1200.00, 10),
(2, 'Desk Chair', 'Furniture', 150.50, 25),
(3, 'Wireless Mouse', 'Electronics', 30.00, 100);

INSERT INTO customers (customer_id, customer_name, email, city, country) VALUES
(1, 'Alice Johnson', 'alice@email.com', 'New York', 'USA'),
(2, 'Bob Smith', 'bob@email.com', 'San Francisco', 'USA');

INSERT INTO orders (order_id, customer_id, order_date, total_price) VALUES
(101, 1, '2024-05-15', 1200.00),
(102, 2, '2024-06-01', 180.50);

### Instructions:
- Generate **valid SQL queries using SQLite syntax**.
- Keep the queries **concise and efficient**.
- If a query requires multiple tables, **use appropriate JOIN operations**.
- If a question cannot be answered based on the given schema, respond with `"The answer is beyond this database."`
""" } ]


In [ ]:
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.

**Example 1**
- **User Question**: "Find the top 5 most expensive products."
- **SQL Output**:
```sql
SELECT product_name, price
FROM products
ORDER BY price DESC
LIMIT 5;
"""
})

In [ ]:
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-4o",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [ ]:
#new
context_user = context.copy()
print(return_CCRMSQL("""how do i find the top salary""", context_user))

The answer is beyond this database.


In [ ]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("how do i find the top salary", old_context_user))

This is your SQL:

```sql
SELECT MAX(salary) AS top_salary FROM salary;
```

This SQL command selects the maximum salary from the salary table, labeling it as top_salary.


In [ ]:
#new
print(return_CCRMSQL("What institution has the highest salary", context_user))

"The answer is beyond this database."


In [ ]:
#old
print(return_CCRMSQL("YOUR QUERY HERE", old_context_user))

This is your SQL:
```sql
SELECT s.Institution
FROM studies s
JOIN salary sa ON s.ID_usr = sa.ID_usr
GROUP BY s.Institution
ORDER BY AVG(sa.salary) DESC
LIMIT 1;
```

This SQL query joins the "studies" and "salary" tables on the ID_usr column. It then calculates the average salary for each institution, orders the results in descending order based on the average salary, and returns the institution with the highest average salary.


# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

# My own version with booksales and few shots:

---



Installing dotenv

In [2]:
pip install python-dotenv


In [4]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

# Context
context = [ {'role':'system', 'content':"""
-- Table Definitions

CREATE TABLE books (
  book_id INT,
  title VARCHAR,
  author_id INT,
  genre VARCHAR,
  year_published INT
);

-- Sample rows
-- (1, '1984', 1, 'Dystopian', 1949)
-- (2, 'Brave New World', 2, 'Dystopian', 1932)
-- (3, 'Animal Farm', 1, 'Satire', 1945)

CREATE TABLE authors (
  author_id INT,
  name VARCHAR,
  nationality VARCHAR
);

-- Sample rows
-- (1, 'George Orwell', 'British')
-- (2, 'Aldous Huxley', 'British')
-- (3, 'Isaac Asimov', 'American')

CREATE TABLE sales (
  sale_id INT,
  book_id INT,
  sale_date DATE,
  price FLOAT
);

-- Sample rows
-- (1, 1, '2022-01-15', 15.99)
-- (2, 2, '2022-01-20', 12.99)
-- (3, 1, '2022-02-10', 14.99)
"""} ]

# Few-shots
context.append({'role': 'system', 'content': """
-- Maintain the SQL order simple and efficient as you can, using valid SQLite.
-- Answer the following questions based on the tables above.

-- Q: What is the title of the most sold book?
SELECT b.title
FROM books b
JOIN sales s ON b.book_id = s.book_id
GROUP BY b.title
ORDER BY COUNT(s.sale_id) DESC
LIMIT 1;

-- Q: How many books did George Orwell write?
SELECT COUNT(*)
FROM books b
JOIN authors a ON b.author_id = a.author_id
WHERE a.name = 'George Orwell';

-- Q: What is the average price of books written by British authors?
SELECT AVG(s.price)
FROM sales s
JOIN books b ON s.book_id = b.book_id
JOIN authors a ON b.author_id = a.author_id
WHERE a.nationality = 'British';
"""})

# Define the function to call the OpenAI API
def return_CCRMSQL(user_message, context):
    client = OpenAI(api_key=OPENAI_API_KEY)
    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})
    response = client.chat.completions.create(
            model="gpt-4o",
            messages=newcontext,
            temperature=0, # using this temperature for more consistent results
        )
    return (response.choices[0].message.content)

# Book sales question
user_query = "Which year had the highest number of book sales?"
print(return_CCRMSQL(user_query, context))


```sql
SELECT strftime('%Y', s.sale_date) AS year, COUNT(s.sale_id) AS sales_count
FROM sales s
GROUP BY year
ORDER BY sales_count DESC
LIMIT 1;
```


# Reflections

I've been playing around with GPT-4o to generate SQL from regular English questions, and it's working pretty good. The AI does a solid job translating what I ask into proper SQL code and gives me structured results that make sense.

The model sometimes struggles to fully understand the context of what I'm asking, gets confused with ambiguous questions, and occasionally the performance could be better.

I'm thinking of making some improvements like:

- Helping the model better understand database schemas
- Adding some validation to double-check the SQL is accurate
- Making the queries more efficient (but i dont know enough about SQL.)

I have set up the database schema, included some sample data, and added a few example queries as few-shot prompts to guide the model. I used gpt-4o with temperature=0 for consistent results. The function structure makes it easy to pass in new questions and get SQL back.